In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from data_prep import clean_data
#from get_stats import read_data_from_files #nbastuffer_dataframe

In [2]:
stats_df = clean_data() # preprocessed, clean data
stats_df

,POS,AGE,GP,MPG,USG%,FTA,FT%,2PA,2P%,3PA,...,TS%,PPG,RPG,APG,SPG,BPG,TPG,VI,ORTG,DRTG
0,G,25.69,31,19.0,12.2,13,0.923,30,0.500,127,...,0.507,5.3,1.5,0.6,0.55,0.19,0.45,3.5,103.1,103.9
1,F,28.51,10,12.3,9.2,10,0.700,3,0.667,15,...,0.379,1.7,2.5,0.8,0.10,0.40,0.40,4.9,87.1,98.5
2,G,22.93,34,12.6,13.5,9,0.778,36,0.361,74,...,0.474,3.2,1.8,1.9,0.38,0.15,0.82,7.0,99.5,108.1
3,C,25.73,80,33.4,16.4,292,0.500,807,0.596,2,...,0.591,13.9,9.5,1.6,1.49,0.96,1.73,7.1,119.9,102.7
5,F,22.19,19,10.2,9.9,4,1.000,13,0.385,23,...,0.424,1.7,1.0,0.3,0.05,0.21,0.32,3.1,85.3,115.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2957,F,20.30,19,11.6,11.0,4,0.750,18,0.500,35,...,0.411,2.4,0.9,0.6,0.40,0.50,0.10,3.8,97.3,113.2
2958,G,25.90,25,10.3,10.9,9,0.667,41,0.488,16,...,0.500,2.4,0.9,1.2,0.40,0.10,0.20,5.3,117.6,114.5
2963,G,24.80,23,12.0,11.6,3,0.667,58,0.431,2,...,0.424,2.3,1.6,1.7,0.30,0.10,0.70,6.1,93.2,114.2
2970,G,25.30,5,13.4,10.0,0,0.000,8,0.500,6,...,0.393,2.2,1.8,1.0,0.80,0.60,0.20,4.7,94.4,98.0


In [3]:
features = list(stats_df)[1:]
X, y = stats_df[features], stats_df['POS']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0, stratify = y)

In [10]:
sc = StandardScaler()
fs = SelectKBest()
svm = SVC()

pipe = Pipeline([('scaler', sc), 
                 ("feature_selection", fs),
                  ("svc", svm)])

param_grid = {'svc__C': [1],
          'feature_selection__k': [4, 6, 8, 10, 12, 20]}

search = GridSearchCV(pipe, param_grid, cv=5)
search.fit(X_train, y_train)
print(search.best_params_)

model = search.best_estimator_
print("Validation accuracy     :", search.best_score_)

y_pred = model.predict(X_test)
print(f"Test data model accuracy: {model.score(X_test, y_test)}")
print("\n", classification_report(y_test, y_pred))

{'feature_selection__k': 20, 'svc__C': 1}
Validation accuracy     : 0.8152523897367097
Test data model accuracy: 0.8222996515679443

               precision    recall  f1-score   support

           C       0.89      0.66      0.76        62
           F       0.75      0.80      0.77       217
           G       0.87      0.87      0.87       295

    accuracy                           0.82       574
   macro avg       0.84      0.78      0.80       574
weighted avg       0.83      0.82      0.82       574

